## Get directory listing with subfolders

In [19]:
import os
import pandas as pd


def list_files_in_directory(directory):
    file_list = []
    
    # Walk through directory and subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
#            file_path = os.path.join(root, file)
            file_name = os.path.basename(file)
            file_type = os.path.splitext(file)[1]
            if file_type in ['.mp4', '.srt', '.vtt']:
                file_path = root
                file_name = os.path.basename(file)
                file_list.append([file_path, file_name, file_type,"",""])
    
    return file_list

def save_to_excel(file_list, output_file):
    # Create a DataFrame
    df = pd.DataFrame(file_list, columns=["directory path", "File Name", "File Type","video file name","vtt file name"])
    
    # Write DataFrame to Excel file
    df.to_excel(output_file, index=False)

# Define the directory you want to scan
directory_to_scan = r"C:\Users\Sean\OneDrive - Virginia Tech\Documents\SGC_Teaching_MSE_2034_D2L\vscode_brightspace_development\brightspace\materials_science_renderings\assets\media"

# Output Excel file path
output_excel_file = os.path.join(directory_to_scan, 'file_list.xlsx')

# Get list of files
file_list = list_files_in_directory(directory_to_scan)

# Save the list to an Excel file
save_to_excel(file_list, output_excel_file)

print(f"File list saved to {output_excel_file}")


File list saved to C:\Users\Sean\OneDrive - Virginia Tech\Documents\SGC_Teaching_MSE_2034_D2L\vscode_brightspace_development\brightspace\materials_science_renderings\assets\media\file_list.xlsx


## The code below creates a new html file based on a template and an excel file which gives information for the location of video files and corresponding *.vtt files.  If you only have an .srt file just give that name instead and it will be converted automatically to .vtt

In [3]:
import csv
import os

def srt_to_vtt(srt_file_path, vtt_file_path):
    """Converts an SRT file to a VTT file, adjusting time stamps."""
    with open(srt_file_path, 'r', encoding='utf-8') as srt_file:
        lines = srt_file.readlines()
    
    with open(vtt_file_path, 'w', encoding='utf-8') as vtt_file:
        vtt_file.write('WEBVTT\n\n')  # Add the required WEBVTT header
        for line in lines:
            if '-->' in line:
                line = line.replace(',', '.')  # Replace comma with period in timestamps
            vtt_file.write(line)


def modify_html(template_path, csv_path, output_dir, base_path):
    """Reads the CSV file, converts SRT to VTT, and modifies HTML template."""
    with open(template_path, 'r', encoding='utf-8') as file:
        template = file.read()
    
    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Paths for video and VTT files
            video_full_path = os.path.join(row['directory path'], row['video file name'])
            vtt_file_name = row['vtt file name']
            vtt_full_path = os.path.join(row['directory path'], vtt_file_name)
            new_vtt_file_path = f"{os.path.splitext(vtt_full_path)[0]}.vtt"

  

            # Check and convert SRT to VTT if needed
            if os.path.exists(vtt_full_path):
                if vtt_full_path.endswith('.srt'):
                    srt_to_vtt(vtt_full_path, new_vtt_file_path)
                    vtt_full_path = new_vtt_file_path  # Update path to use the new VTT file

                # Compute the relative paths
                relative_video_path = os.path.relpath(video_full_path, base_path)
                relative_vtt_path = os.path.relpath(vtt_full_path, base_path)
                modified_html = template.replace('../assets/media/filename.mp4', relative_video_path.replace('\\', '/'))
                modified_html = modified_html.replace('../assets/media/filename.vtt', relative_vtt_path.replace('\\', '/'))
           
            else:
                # VTT file does not exist, create HTML without VTT link
                relative_video_path = os.path.relpath(video_full_path, base_path)
                modified_html = template.replace('../assets/media/filename.mp4', relative_video_path.replace('\\', '/'))
                modified_html = modified_html.replace('type="text/vtt" src="../assets/media/filename.vtt"', '')  # Remove VTT link

            # Output HTML file path
            video_file_base = os.path.splitext(row['video file name'])[0]
            output_html_path = os.path.join(output_dir, f"{video_file_base}.html")

            # Write the modified HTML to a new file
            with open(output_html_path, 'w', encoding='utf-8') as output_file:
                output_file.write(modified_html)
            print(f"Generated {output_html_path}")

# Usage example (ensure these paths are accurate for your setup)
template_path = r"C:\Users\Sean\OneDrive - Virginia Tech\Documents\SGC_Teaching_MSE_2034_D2L\vscode_brightspace_development\brightspace\materials_science_renderings\lessons\template_video.html"
csv_path = r"C:\Users\Sean\OneDrive - Virginia Tech\Documents\SGC_Teaching_MSE_2034_D2L\vscode_brightspace_development\brightspace\materials_science_renderings\assets\media\input.csv"
output_dir = r"C:\Users\Sean\OneDrive - Virginia Tech\Documents\SGC_Teaching_MSE_2034_D2L\vscode_brightspace_development\brightspace\materials_science_renderings\lessons\video_lessons"
base_path = output_dir
modify_html(template_path, csv_path, output_dir, base_path)


Generated C:\Users\Sean\OneDrive - Virginia Tech\Documents\SGC_Teaching_MSE_2034_D2L\vscode_brightspace_development\brightspace\materials_science_renderings\lessons\video_lessons\00.01.welcome_intro.html
Generated C:\Users\Sean\OneDrive - Virginia Tech\Documents\SGC_Teaching_MSE_2034_D2L\vscode_brightspace_development\brightspace\materials_science_renderings\lessons\video_lessons\01.01_intro.html
Generated C:\Users\Sean\OneDrive - Virginia Tech\Documents\SGC_Teaching_MSE_2034_D2L\vscode_brightspace_development\brightspace\materials_science_renderings\lessons\video_lessons\02.01_bonding.html
Generated C:\Users\Sean\OneDrive - Virginia Tech\Documents\SGC_Teaching_MSE_2034_D2L\vscode_brightspace_development\brightspace\materials_science_renderings\lessons\video_lessons\02.02_bonding.html
Generated C:\Users\Sean\OneDrive - Virginia Tech\Documents\SGC_Teaching_MSE_2034_D2L\vscode_brightspace_development\brightspace\materials_science_renderings\lessons\video_lessons\03.01_structures.html
Gen